In [276]:
import json
import pandas as pd
pd.options.plotting.backend = "plotly"
import numpy as np
# import plotly.express as px
import powerwall
import voltwatt


## Test Profiles

In [277]:

pw = powerwall.Powerwall() # create EUT 

vw1 = voltwatt.VoltWattTest(1, pw)  # create test profiles
vw2 = voltwatt.VoltWattTest(2, pw)  # enter test characteristic (1,2,3)
vw3 = voltwatt.VoltWattTest(3, pw)

# make dataframe to hold test profiles
df1 = pd.DataFrame.from_dict(vw1.__dict__, orient='index')
df2 = pd.DataFrame.from_dict(vw2.__dict__, orient='index')
df3 = pd.DataFrame.from_dict(vw3.__dict__, orient='index')

VW = pd.DataFrame()
VW['vw.1'], VW['vw.2'], VW['vw.3'] = df1[0], df2[0], df3[0]
print(VW)
VW.to_csv('out.csv')


                  vw.1    vw.2    vw.3
characteristic     1.0     2.0     3.0
v1               254.4   252.0   261.6
p1              5000.0  5000.0  5000.0
v2               264.0   264.0   264.0
p2_prime       -5000.0 -5000.0 -5000.0
ol_tr             10.0    90.0     0.5


## Calculate Watts as Function of Volts

In [278]:
# run voltage vector through volt watt function
data_length = 100
voltage_range_min = 230
voltage_range_max = 280

v = np.linspace(voltage_range_min, voltage_range_max, data_length)
wat1 = np.zeros(data_length)

for count, voltage in enumerate(v):

    wat1[count] = vw1.watts_from_volts(v[count])



## Ideal Volt Watt Curve Data

In [279]:

volt_watt_data = {'volts': v.T , 'watts': wat1.T}
vwdf = pd.DataFrame.from_dict(volt_watt_data)


## Simple Plotter

In [280]:

fig = vwdf.plot(
    x = 'volts',
    y = 'watts',
    title = 'Volt-Watt Curve'   
)
fig.show()


## Tolerance range for test variables

In [281]:
watt_tolerance = pw.prated * 0.05
vwdf['w-hi'] = vwdf['watts'] + watt_tolerance
vwdf['w-lo'] = vwdf['watts'] - watt_tolerance

print(vwdf.head(2))


        volts   watts    w-hi    w-lo
0  230.000000  5000.0  5250.0  4750.0
1  230.505051  5000.0  5250.0  4750.0


## Plot with Tolerances

In [282]:
fig = vwdf.plot(
    x = 'volts',
    y = ['watts','w-hi','w-lo'],
    
    title = 'Volt-Watt Curve'
    #template="simple_white",
    #labels=dict(index="voltage", value="power", variable="Characteristic")
)
fig.show()


In [284]:
#import plotly.graph_objs as go

df = vwdf
xvals = vwdf['volts']
yvals = vwdf['watts'] 

fig = go.Figure([
    go.Scatter(
        name='Expected P',
        x=xvals,
        y=yvals,
        mode='lines',
        line=dict(color='rgb(31, 119, 180)'),
    ),
    go.Scatter(
        name='Upper Bound',
        x=xvals,
        y=yvals+watt_tolerance,
        mode='lines',
        marker=dict(color="#444"),
        line=dict(width=0),
        showlegend=False
    ),
    go.Scatter(
        name='Lower Bound',
        x=xvals,
        y=yvals-watt_tolerance,
        marker=dict(color="#444"),
        line=dict(width=0),
        mode='lines',
        fillcolor='rgba(68, 68, 68, 0.3)',
        fill='tonexty',
        showlegend=False
    )
])
fig.update_layout(
    yaxis_title='Real Power (W)',
    xaxis_title='Grid Voltage (V)',
    title = 'Volt-Watt Curve (characteristic 1)',
    hovermode="x"
)
fig.show()
fig.write_html("plot.html")